# TRABAJO FIN DE MASTER - Propuesta de datos
# **Author**: Cristóbal León-Salas
# **Date**: 2025-07-15

# LIBRERIAS

Se cargan las siguietnes librerias:

-  pandas --> Para tablas de datos.
-  numpy --> para cálculos numéricos y para trabajar con matrices y vectores.
-  os --> Para trabajar con directorios, archivos, carpetas,...
-  matplotlib --> Para hacer visualizaciones gráficas básicas.
-  seaborn --> Para gráficos estadísticos más profesionales y de fácil interpretación.
-  warnings --> Para evitar mensajes de advertencias
-  product -->  Para sacar todas las combinaciones posibles entre los elementos de dos o más listas


In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from itertools import product
from IPython.display import display
# Ignorar el SettingWithCopyWarning
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
sns.set()

# FUNCIONES

## FUNCIÓN "process_dataset"

La función "process_dataset" me va a permitir:

1. Crear una nueva variable llamada DIA.
2. Limpia los datos reemplazando valores inválidos por NaN.
3. Transponer todas las variables D del dataset original a una misma columna.
   

In [2]:
def process_dataset(filename):
    df = pd.read_csv(filename,sep=';') # Leo los datos del archivo y separo por ";"
    for i in range(1,32): # En Python significa del 1 al 31 (1 observación por día), porque range excluye el valor final.
        var_D = f'D{str(i).zfill(2)}'
        var_V = f'V{str(i).zfill(2)}'
        
        # Los valores de "D" que tengan en su variable "V" una "N", será considerados como NAs
        df.loc[df[var_V]=="N", var_D] = np.nan
        
    # Se transponen las variables "D"
    var_list = [ f'D{str(i).zfill(2)}' for i in range(1, 32)] # Vector con todos los dias de un mes: var_list = ['D01', 'D02', 'D03', ..., 'D31']
    df_new = df.melt (id_vars = ["ESTACION","MAGNITUD","PUNTO_MUESTREO","ANO","MES"], value_vars=var_list) #Se crea nuevo dataframe con alguans de las variables definidas. convierte un DataFrame de formato ancho a formato largo, reorganizando columnas en filas.
    
    # Renombra columnas
    df_new.rename(columns={'variable': 'DIA', 'value': 'MEDICION'}, inplace=True)

    return df_new

## FUNCIÓN "es_fecha_valida"

La función "es_fecha_valida" me permita saber si una fecha es válida, con el objeto de eliminar, por ejemplo, las medidas hechas en el 30 de febrero.

In [3]:
def es_fecha_valida(ano, mes, dia):
    try:
        pd.to_datetime(f'{ano}-{mes}-{dia}', format='%Y-%m-%d')
        return True
    except ValueError:
        return False

# DATOS CALIDAD AIRE MADRID

## ITERACIÓN CSV

Se iteran todos los csv para ponerlos en el mismo formato, esto es, con las variables DIA en una misma columna

Leo los raw data sobre la calidad del aire de la ciudad de Madrid tomados en tramos diarios por medio de este link: https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=aecb88a7e2b73410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

In [4]:
# Leo los raw_data de :
df_total = pd.read_parquet('99_01_DATASET_INICIAL.parquet', engine='pyarrow')

## VARIABLE MUESTREO. TRANSFORMACIÓN EN TECNICA

La variable PUNTO_MUESTREO contiene mucha inforamción, mucha de ella duplicada por contener información de las variables ESTACION y MAGNITUD. Saco solo la información de la técnica usada

In [5]:
# Trato la variable PUNTO_MUESTREO y la renombro como TECNICA, por lo que solo me quedaré con el código técnica:
df_total["TECNICA"] = df_total["PUNTO_MUESTREO"].astype(str).str.split("_").str[-1]

# Elimino la columna original PUNTO_MUESTREO
df_total = df_total.drop(columns=["PUNTO_MUESTREO"])

Categorizo las variables ESTTACION, MAGNITUD y TECNICA

In [6]:
df_total["ESTACION"] = df_total["ESTACION"].astype("category")
df_total["MAGNITUD"] = df_total["MAGNITUD"].astype("category")
df_total["TECNICA"] = df_total["TECNICA"].astype("category")

## DATAFRAME GENERADO (df_total)

Saco información del dataframe generado

In [7]:
display(df_total.head())

,ESTACION,MAGNITUD,MEDICION,DIA/MES/ANO,TECNICA
0,4,1,17.0,2001-01-01,38
1,4,6,0.8,2001-01-01,48
2,4,7,45.0,2001-01-01,8
3,4,8,58.0,2001-01-01,8
4,4,12,127.0,2001-01-01,8


In [8]:
display(df_total.tail())

,ESTACION,MAGNITUD,MEDICION,DIA/MES/ANO,TECNICA
1035504,8,35,0.3,2025-06-30,59
1035505,8,6,NaN,2025-06-30,48
1035506,8,7,0.0,2025-06-30,8
1035507,8,8,16.0,2025-06-30,8
1035508,8,9,13.0,2025-06-30,47


In [9]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035509 entries, 0 to 1035508
Data columns (total 5 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   ESTACION     1035509 non-null  category      
 1   MAGNITUD     1035509 non-null  category      
 2   MEDICION     1015726 non-null  float64       
 3   DIA/MES/ANO  1035509 non-null  datetime64[ns]
 4   TECNICA      1035509 non-null  category      
dtypes: category(3), datetime64[ns](1), float64(1)
memory usage: 18.8 MB


In [10]:
df_total.describe(include='all')

,ESTACION,MAGNITUD,MEDICION,DIA/MES/ANO,TECNICA
count,1035509.0,1035509.0,1.015726e+06,1035509,1035509
unique,24.0,14.0,NaN,NaN,8
top,24.0,8.0,NaN,NaN,8
freq,109272.0,167446.0,NaN,NaN,502338
mean,NaN,NaN,3.003520e+01,2014-05-10 17:32:44.484326400,NaN
min,NaN,NaN,0.000000e+00,2001-01-01 00:00:00,NaN
25%,NaN,NaN,2.600000e+00,2009-12-09 00:00:00,NaN
50%,NaN,NaN,1.600000e+01,2014-08-15 00:00:00,NaN
75%,NaN,NaN,4.200000e+01,2019-04-26 00:00:00,NaN
max,NaN,NaN,1.031000e+03,2025-06-30 00:00:00,NaN


## DATAFRAME COMPLETO

De este dataframe generado, observo que faltan muchos valores, ya que:

1. Para cada día debe haber una medición con todas las combinaciones posibles de Estación, Magnitud y Técnica.
2. Porque en la variable DIA/MES/ANO faltan días para medir, por ejemplo todos los dias de diciembre de 2024.

Por estas dos razones, procedo a indicar en la variable MEDICIÓN NAs en todas aquellas combinaciones donde, por el motivo que sea, no he tenido registro

In [11]:
# Cargar datos ya preprocesados
df_prepro = df_total
# parse_dates = convierte automáticamente esa columna a formato de fecha (datetime64[ns]).
# dayfirst = primer número en las fechas es el día.

# Crear rango de fechas completo
fecha_min = df_prepro["DIA/MES/ANO"].min()
fecha_max = df_prepro["DIA/MES/ANO"].max()
fechas_completas = pd.date_range(start=fecha_min, end=fecha_max, freq="D")

# Obtener niveles únicos de las variables declaradas como categóricas anteriormente
estaciones = df_prepro["ESTACION"].unique()
magnitudes = df_prepro["MAGNITUD"].unique()
tecnicas = df_prepro["TECNICA"].unique()

# Producto cartesiano de combinaciones
combinaciones = list(product(estaciones, magnitudes, tecnicas, fechas_completas))

# Crear DataFrame con combinaciones
df_completo = pd.DataFrame(combinaciones, columns=["ESTACION", "MAGNITUD", "TECNICA", "DIA/MES/ANO"])

# Convertir las variables ESTACIÓN, MAGNITUD y TECNICA de nuevo a categóricas y la variable DIA/MES/ANO de nuevo a temporal
df_completo["ESTACION"] = df_completo["ESTACION"].astype("category")
df_completo["MAGNITUD"] = df_completo["MAGNITUD"].astype("category")
df_completo["TECNICA"] = df_completo["TECNICA"].astype("category")
df_completo["DIA/MES/ANO"] = pd.to_datetime(df_completo["DIA/MES/ANO"])


In [12]:
display(df_completo.head())

,ESTACION,MAGNITUD,TECNICA,DIA/MES/ANO
0,4,1,38,2001-01-01
1,4,1,38,2001-01-02
2,4,1,38,2001-01-03
3,4,1,38,2001-01-04
4,4,1,38,2001-01-05


In [13]:
display(df_completo.tail())

,ESTACION,MAGNITUD,TECNICA,DIA/MES/ANO
24049531,48,9,A,2025-06-26
24049532,48,9,A,2025-06-27
24049533,48,9,A,2025-06-28
24049534,48,9,A,2025-06-29
24049535,48,9,A,2025-06-30


In [14]:
df_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24049536 entries, 0 to 24049535
Data columns (total 4 columns):
 #   Column       Dtype         
---  ------       -----         
 0   ESTACION     category      
 1   MAGNITUD     category      
 2   TECNICA      category      
 3   DIA/MES/ANO  datetime64[ns]
dtypes: category(3), datetime64[ns](1)
memory usage: 252.3 MB


In [15]:
df_completo.describe(include='all')

,ESTACION,MAGNITUD,TECNICA,DIA/MES/ANO
count,24049536.0,24049536.0,24049536,24049536
unique,24.0,14.0,8,NaN
top,4.0,1.0,2,NaN
freq,1002064.0,1717824.0,3006192,NaN
mean,NaN,NaN,NaN,2013-04-01 00:00:00
min,NaN,NaN,NaN,2001-01-01 00:00:00
25%,NaN,NaN,NaN,2007-02-15 00:00:00
50%,NaN,NaN,NaN,2013-04-01 00:00:00
75%,NaN,NaN,NaN,2019-05-17 00:00:00
max,NaN,NaN,NaN,2025-06-30 00:00:00


Incluyo los valores de MEDICION conocidos en el dataframe df_completo:

In [16]:
# Unir los dataframes para añadir la columna MEDICION
df_final = pd.merge(df_completo, df_total, on=["ESTACION", "MAGNITUD", "TECNICA", "DIA/MES/ANO"], how="left")


In [17]:
display(df_final.head())

,ESTACION,MAGNITUD,TECNICA,DIA/MES/ANO,MEDICION
0,4,1,38,2001-01-01,17.0
1,4,1,38,2001-01-02,15.0
2,4,1,38,2001-01-03,15.0
3,4,1,38,2001-01-04,15.0
4,4,1,38,2001-01-05,16.0


In [18]:
display(df_final.tail())

,ESTACION,MAGNITUD,TECNICA,DIA/MES/ANO,MEDICION
24049531,48,9,A,2025-06-26,NaN
24049532,48,9,A,2025-06-27,NaN
24049533,48,9,A,2025-06-28,NaN
24049534,48,9,A,2025-06-29,NaN
24049535,48,9,A,2025-06-30,NaN


In [19]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24049536 entries, 0 to 24049535
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   ESTACION     category      
 1   MAGNITUD     category      
 2   TECNICA      category      
 3   DIA/MES/ANO  datetime64[ns]
 4   MEDICION     float64       
dtypes: category(3), datetime64[ns](1), float64(1)
memory usage: 435.8 MB


In [20]:
df_final.describe(include='all')

,ESTACION,MAGNITUD,TECNICA,DIA/MES/ANO,MEDICION
count,24049536.0,24049536.0,24049536,24049536,1.015726e+06
unique,24.0,14.0,8,NaN,NaN
top,4.0,1.0,2,NaN,NaN
freq,1002064.0,1717824.0,3006192,NaN,NaN
mean,NaN,NaN,NaN,2013-04-01 00:00:00,3.003520e+01
min,NaN,NaN,NaN,2001-01-01 00:00:00,0.000000e+00
25%,NaN,NaN,NaN,2007-02-15 00:00:00,2.600000e+00
50%,NaN,NaN,NaN,2013-04-01 00:00:00,1.600000e+01
75%,NaN,NaN,NaN,2019-05-17 00:00:00,4.200000e+01
max,NaN,NaN,NaN,2025-06-30 00:00:00,1.031000e+03


Verifico que tengo todas las combinaciones, viendo si coinciden los 24,049,536 filas, con este cálculo

In [21]:
n_estaciones = df_final['ESTACION'].nunique()
n_magnitudes = df_final['MAGNITUD'].nunique()
n_tecnicas = df_final['TECNICA'].nunique()
n_dias = df_final['DIA/MES/ANO'].nunique()

total_esperado = n_estaciones * n_magnitudes * n_tecnicas * n_dias
print("Total esperado:", total_esperado)
print("Total en df_final:", len(df_final))

Total esperado: 24049536
Total en df_final: 24049536


Se observa que es correcto.

## % NAs en variable MEDICIÓN

Observo que el número de instancias de la variable medición es más bajo que el del resto de las variables, por lo que me hace pensar que haya mucho valores NAs. Saco este número de NAs

In [22]:
na_med = int(df_final["MEDICION"].isna().sum())
print(na_med)

23033810


In [23]:
perc_NA_med = na_med / len(df_final) * 100
print(f"{perc_NA_med:.2f}%")

95.78%


Casi un 96% de las instancias están vacias. Es por ello por lo que voy a hacer un análisis de cuales son las combinacioens de "ESTACION", "MAGNITUD" y "TECNICA"  que más valores de la variable medición aportan al dataframe

## ANÁLISIS COMBINACIONES ESTACION, MAGNITUD, TECNICA

In [24]:

# Número total de días únicos
total_dias = df_final['DIA/MES/ANO'].nunique()

# Cálculo del conteo y porcentaje
conteo_mediciones = (
    df_final
    .dropna(subset=["MEDICION"])  # Filtrar solo las filas con mediciones
    .groupby(["ESTACION", "MAGNITUD", "TECNICA"], observed=False) # Agrupo por combinación de valores de las variables "ESTACION", "MAGNITUD" Y "TECINCA"
    .size() # Obtengo el número de instancias por cada combinación
    .reset_index(name="N_mediciones") # Convierte el objeto GroupBy en un DataFrame plano
    .sort_values(by="N_mediciones", ascending=False) # Ordeno la tabla por número de instancias de cada combinación
)

# Añadir columna con el porcentaje respecto al total de días
conteo_mediciones["% días con valor MEDICION"] = (
    conteo_mediciones["N_mediciones"] / total_dias * 100
).round(2)  # Redondeamos a 2 decimales


In [25]:
conteo_mediciones.describe(include='all')

,ESTACION,MAGNITUD,TECNICA,N_mediciones,% días con valor MEDICION
count,2688.0,2688.0,2688,2688.000000,2688.000000
unique,24.0,14.0,8,NaN,NaN
top,4.0,1.0,2,NaN,NaN
freq,112.0,192.0,336,NaN,NaN
mean,NaN,NaN,NaN,377.874256,4.223460
std,NaN,NaN,NaN,1568.648591,17.532602
min,NaN,NaN,NaN,0.000000,0.000000
25%,NaN,NaN,NaN,0.000000,0.000000
50%,NaN,NaN,NaN,0.000000,0.000000
75%,NaN,NaN,NaN,0.000000,0.000000


In [26]:
display(conteo_mediciones.head(10))

,ESTACION,MAGNITUD,TECNICA,N_mediciones,% días con valor MEDICION
1366,40,7,8,8549,95.55
1398,40,12,8,8549,95.55
1374,40,8,8,8549,95.55
1174,38,12,8,8543,95.48
1150,38,8,8,8543,95.48
1142,38,7,8,8543,95.48
1062,36,12,8,8533,95.37
1030,36,7,8,8533,95.37
1038,36,8,8,8533,95.37
246,11,7,8,8517,95.19


In [27]:
display(conteo_mediciones.tail(10))

,ESTACION,MAGNITUD,TECNICA,N_mediciones,% días con valor MEDICION
6,4,1,8,0,0.0
7,4,1,A,0,0.0
8,4,6,2,0,0.0
9,4,6,38,0,0.0
10,4,6,47,0,0.0
75,4,30,48,0,0.0
12,4,6,59,0,0.0
13,4,6,6,0,0.0
14,4,6,8,0,0.0
15,4,6,A,0,0.0


Observo muchas combinaciones que no tienen valores MEDICION en todo el dataframe final. Elimino estas combinaciones

In [28]:
# Filtramos combinaciones con al menos una medición válida
combinaciones_validas = (
    df_final
    .dropna(subset=["MEDICION"])
    .groupby(["ESTACION", "MAGNITUD", "TECNICA"], observed=True) # A diferencia de antes, cambio el observed por True
    .size()
    .reset_index()[["ESTACION", "MAGNITUD", "TECNICA"]]
)

In [29]:
# Hacemos un inner merge para conservar solo las combinaciones válidas
df_final = df_final.merge(combinaciones_validas, on=["ESTACION", "MAGNITUD", "TECNICA"], how="inner") # Un inner join mantiene solo las filas que tienen coincidencia exacta en las columnas clave (en este caso, "ESTACION", "MAGNITUD" y "TECNICA") en ambos dataframes.

Vuelvo a analizar que % de instancias NAs hay ahora

In [30]:
na_med = int(df_final["MEDICION"].isna().sum())
print(na_med)

442635


In [31]:
perc_NA_med = na_med / len(df_final) * 100
print(f"{perc_NA_med:.2f}%")

30.35%


Un 30.35% de registros vacíos empieza a ser un valor considerablemente bueno para empezar un análisis.

In [32]:
display(df_final.head())

,ESTACION,MAGNITUD,TECNICA,DIA/MES/ANO,MEDICION
0,4,1,38,2001-01-01,17.0
1,4,1,38,2001-01-02,15.0
2,4,1,38,2001-01-03,15.0
3,4,1,38,2001-01-04,15.0
4,4,1,38,2001-01-05,16.0


In [33]:
display(df_final.tail())

,ESTACION,MAGNITUD,TECNICA,DIA/MES/ANO,MEDICION
1458356,48,9,47,2025-06-26,NaN
1458357,48,9,47,2025-06-27,24.0
1458358,48,9,47,2025-06-28,25.0
1458359,48,9,47,2025-06-29,NaN
1458360,48,9,47,2025-06-30,NaN


In [34]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458361 entries, 0 to 1458360
Data columns (total 5 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   ESTACION     1458361 non-null  category      
 1   MAGNITUD     1458361 non-null  category      
 2   TECNICA      1458361 non-null  category      
 3   DIA/MES/ANO  1458361 non-null  datetime64[ns]
 4   MEDICION     1015726 non-null  float64       
dtypes: category(3), datetime64[ns](1), float64(1)
memory usage: 26.4 MB


In [35]:
df_final.describe(include='all')

,ESTACION,MAGNITUD,TECNICA,DIA/MES/ANO,MEDICION
count,1458361.0,1458361.0,1458361,1458361,1.015726e+06
unique,24.0,14.0,8,NaN,NaN
top,8.0,8.0,8,NaN,NaN
freq,125258.0,214728.0,644184,NaN,NaN
mean,NaN,NaN,NaN,2013-03-31 23:59:59.999999488,3.003520e+01
min,NaN,NaN,NaN,2001-01-01 00:00:00,0.000000e+00
25%,NaN,NaN,NaN,2007-02-15 00:00:00,2.600000e+00
50%,NaN,NaN,NaN,2013-04-01 00:00:00,1.600000e+01
75%,NaN,NaN,NaN,2019-05-17 00:00:00,4.200000e+01
max,NaN,NaN,NaN,2025-06-30 00:00:00,1.031000e+03


## NORMALIZACIÓN VARIABLE MAGNITUD

En la variable MAGNITUD, algunos valores vienen expresados en mg/m3 y otros en μg/m3, por ello, normalizo la variable para que todos sus valores queden expresados en la misma unidad.

In [36]:
df_final['unidad'] = df_final['MAGNITUD'].isin([1, 7, 8, 9, 10, 12, 14, 20, 30, 35, 37, 38, 39]) # Se crea una nueva columna "unidad" con valores booleanos (True o False), indicando si esa magnitud ya está en µg/m³

In [37]:
df_final['MEDICION_ugm3'] = df_final.apply(lambda x: x['MEDICION'] * 1000 if not x['unidad'] else x['MEDICION'], axis=1)

# axis = 1 --> Aplicado a cada fila.
#  x['MEDICION'] * 1000 if not x['unidad'] else x['MEDICION'] --> Si la variable unidad es False, multiplica el valor de MEDICION por 1000 (estaba en mg/m³), si no, deja el valor tal cual (ya está en μg/m³).

Compruebo:

In [38]:
display(df_final.head())

,ESTACION,MAGNITUD,TECNICA,DIA/MES/ANO,MEDICION,unidad,MEDICION_ugm3
0,4,1,38,2001-01-01,17.0,True,17.0
1,4,1,38,2001-01-02,15.0,True,15.0
2,4,1,38,2001-01-03,15.0,True,15.0
3,4,1,38,2001-01-04,15.0,True,15.0
4,4,1,38,2001-01-05,16.0,True,16.0


Con MAGNITUD = 1 (Dióxido de Azufre), los valores de MEDICIÓN no cambian

In [39]:
display(df_final[df_final['MAGNITUD'] == 6].head())

,ESTACION,MAGNITUD,TECNICA,DIA/MES/ANO,MEDICION,unidad,MEDICION_ugm3
8947,4,6,48,2001-01-01,0.8,False,800.0
8948,4,6,48,2001-01-02,0.5,False,500.0
8949,4,6,48,2001-01-03,0.6,False,600.0
8950,4,6,48,2001-01-04,0.5,False,500.0
8951,4,6,48,2001-01-05,1.0,False,1000.0


Con MAGNITUD = 6 (Monóxido de Carbono), los valores de MEDICIÓN se multiplican por 1000 para que se muestren en la misma unidad que el resto.

Elimino las variables MEDICION, para quedarme con la variable normalizada y unidad

In [40]:
df_final = df_final.drop(columns=["MEDICION", "unidad"])

In [41]:
display(df_final.head())

,ESTACION,MAGNITUD,TECNICA,DIA/MES/ANO,MEDICION_ugm3
0,4,1,38,2001-01-01,17.0
1,4,1,38,2001-01-02,15.0
2,4,1,38,2001-01-03,15.0
3,4,1,38,2001-01-04,15.0
4,4,1,38,2001-01-05,16.0


In [42]:
df_final["MAGNITUD"] = df_final["MAGNITUD"].astype("category")

In [43]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458361 entries, 0 to 1458360
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   ESTACION       1458361 non-null  category      
 1   MAGNITUD       1458361 non-null  category      
 2   TECNICA        1458361 non-null  category      
 3   DIA/MES/ANO    1458361 non-null  datetime64[ns]
 4   MEDICION_ugm3  1015726 non-null  float64       
dtypes: category(3), datetime64[ns](1), float64(1)
memory usage: 26.4 MB


# DATOS METEOROLÓGICOS

## LECTURA DE DATOS

In [44]:
# Cargar datos meteorológicos
df_meteo_raw = pd.read_csv("97_25.07.19_DATOS METEOROLÓGICOS.csv", header=None)

# Seleccionamos las filas 8 y 9 como strings
fila_8 = df_meteo_raw.iloc[8].astype(str)
fila_9 = df_meteo_raw.iloc[9].astype(str)

# Combinamos las dos filas en una sola lista de nombres de columna
nuevos_headers = fila_9 + "_" + fila_8

# Asignamos como nuevos encabezados
df_meteo_raw.columns = nuevos_headers

# Eliminamos las filas anteriores (hasta la 9 inclusive)
df_meteo = df_meteo_raw.iloc[10:].reset_index(drop=True)
df_meteo.columns = nuevos_headers

## VARIABLE TEMPORAL

Procedo a dejar la variable temporal en el mismo formato que el otro dataframe para que luego puedan combinarse

In [45]:
# Extraer los primeros 8 caracteres de 'timestamp' y convertirlos a datetime
df_meteo["DIA/MES/ANO"] = pd.to_datetime(df_meteo["timestamp_aggregation"].str[:8], format="%Y%m%d")

# Eliminar columna original si no se necesita
df_meteo = df_meteo.drop(columns=["timestamp_aggregation"])

## RENOMBRAR VARIABLES

In [46]:

# Diccionario: clave = nombre original de columna, valor = nombre nuevo
magnitudes_dict = {
    "Madrid Temperature [2 m elevation corrected]_Maximum": "TEMPERATURA_MAX (°C)",
    "Madrid Temperature [2 m elevation corrected]_Minimum": "TEMPERATURA_MIN (°C)",
    "Madrid Temperature [2 m elevation corrected]_Mean": "TEMPERATURA_MED (°C)",
    "Madrid Precipitation Total_Summation": "PRECIPITACIONES (mm)",
    "Madrid Relative Humidity [2 m]_Maximum": "HUMEDAD_MAX (%)",
    "Madrid Relative Humidity [2 m]_Minimum": "HUMEDAD_MIN (%)",
    "Madrid Relative Humidity [2 m]_Mean": "HUMEDAD_MED (%)",
    "Madrid Snowfall Amount_Summation": "NIEVE (cm)",
    "Madrid Wind Speed [10 m]_Maximum": "VIENTO_MAX_10 (km/h)",
    "Madrid Wind Speed [10 m]_Minimum": "VIENTO_MIN_10 (km/h)",
    "Madrid Wind Speed [10 m]_Mean": "VIENTO_MED_10 (km/h)",
    "Madrid Wind Direction Dominant [10 m]_nan": "VIENTO_DIR_10 (°)",
    "Madrid Wind Speed [100 m]_Maximum": "VIENTO_MAX_100 (km/h)",
    "Madrid Wind Speed [100 m]_Minimum": "VIENTO_MIN_100 (km/h)",
    "Madrid Wind Speed [100 m]_Mean": "VIENTO_MED_100 (km/h)",
    "Madrid Wind Direction Dominant [100 m]_nan": "VIENTO_DIR_100 (°)",
    "Madrid Cloud Cover Total_Mean": "NUBES (%)",
    "Madrid Sunshine Duration_Summation": "HORAS_SOL (min)",
    "Madrid Shortwave Radiation_Summation": "RADIACION_SOLAR (W/mÂ²)",
    "Madrid Mean Sea Level Pressure [MSL]_Maximum": "PRESION_MAX (hPa)",
    "Madrid Mean Sea Level Pressure [MSL]_Minimum": "PRESION_MIN (hPa)",
    "Madrid Mean Sea Level Pressure [MSL]_Mean": "PRESION_MED (hPa)"
}

# Renombrar columnas
df_meteo.rename(columns=magnitudes_dict, inplace=True)


## CONVERTIR A VARIABLES NUMÉRICAS

In [47]:
# Seleccionar columnas numéricas (todas menos la fecha)
columnas_a_convertir = df_meteo.columns.difference(["DIA/MES/ANO"])

# Convertir a numérico (float) forzando NaN en errores
for col in columnas_a_convertir:
    df_meteo[col] = pd.to_numeric(df_meteo[col], errors='coerce')

In [48]:
df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8947 entries, 0 to 8946
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   TEMPERATURA_MAX (°C)     8946 non-null   float64       
 1   TEMPERATURA_MIN (°C)     8946 non-null   float64       
 2   TEMPERATURA_MED (°C)     8946 non-null   float64       
 3   PRECIPITACIONES (mm)     8946 non-null   float64       
 4   HUMEDAD_MAX (%)          8946 non-null   float64       
 5   HUMEDAD_MIN (%)          8946 non-null   float64       
 6   HUMEDAD_MED (%)          8946 non-null   float64       
 7   NIEVE (cm)               8946 non-null   float64       
 8   VIENTO_MAX_10 (km/h)     8946 non-null   float64       
 9   VIENTO_MIN_10 (km/h)     8946 non-null   float64       
 10  VIENTO_MED_10 (km/h)     8946 non-null   float64       
 11  VIENTO_DIR_10 (°)        8946 non-null   float64       
 12  VIENTO_MAX_100 (km/h)    8946 non-

In [49]:
display(df_meteo.head())

,TEMPERATURA_MAX (°C),TEMPERATURA_MIN (°C),TEMPERATURA_MED (°C),PRECIPITACIONES (mm),HUMEDAD_MAX (%),HUMEDAD_MIN (%),HUMEDAD_MED (%),NIEVE (cm),VIENTO_MAX_10 (km/h),VIENTO_MIN_10 (km/h),...,VIENTO_MIN_100 (km/h),VIENTO_MED_100 (km/h),VIENTO_DIR_100 (°),NUBES (%),HORAS_SOL (min),RADIACION_SOLAR (W/mÂ²),PRESION_MAX (hPa),PRESION_MIN (hPa),PRESION_MED (hPa),DIA/MES/ANO
0,11.073577,6.123577,8.805244,3.600000,92.971060,85.407870,89.648130,0.0,25.068514,9.779817,...,19.211996,30.012077,202.18532,86.166664,24.510103,620.0,1016.5,1006.2,1010.35004,2001-01-01
1,10.763577,6.703577,8.393994,0.000000,94.317500,58.461136,80.099490,0.0,23.667091,11.720751,...,22.668570,29.593008,227.16107,55.375000,327.718200,1942.0,1018.8,1009.3,1012.97090,2001-01-02
2,9.553577,6.373577,8.138577,2.800000,93.021324,77.196846,86.401764,0.0,22.702845,10.188700,...,18.356470,27.256058,215.83910,86.666664,35.062290,862.0,1021.0,1015.5,1018.71674,2001-01-03
3,12.503577,8.243577,10.381910,0.100000,94.347970,62.965412,81.007430,0.0,21.638964,12.475961,...,18.075441,27.357481,225.98854,57.791668,294.457580,1685.0,1020.4,1013.9,1018.04980,2001-01-04
4,14.563578,7.933577,12.401077,10.000001,95.628430,87.159270,91.249020,0.0,28.467327,13.570615,...,26.099379,36.826267,209.32076,96.625000,4.242425,435.0,1018.2,1005.5,1009.74580,2001-01-05


# COMBINACION AMBAS BASES DE DATOS

Se fusionan ambas bases de datos tomando como variable común la fecha

In [50]:
df_final = df_final.merge(df_meteo, on="DIA/MES/ANO", how="left")

In [51]:
df_final.to_parquet("01_DATASET_FINAL.parquet", index=False)

In [52]:
display(df_final.head())

,ESTACION,MAGNITUD,TECNICA,DIA/MES/ANO,MEDICION_ugm3,TEMPERATURA_MAX (°C),TEMPERATURA_MIN (°C),TEMPERATURA_MED (°C),PRECIPITACIONES (mm),HUMEDAD_MAX (%),...,VIENTO_MAX_100 (km/h),VIENTO_MIN_100 (km/h),VIENTO_MED_100 (km/h),VIENTO_DIR_100 (°),NUBES (%),HORAS_SOL (min),RADIACION_SOLAR (W/mÂ²),PRESION_MAX (hPa),PRESION_MIN (hPa),PRESION_MED (hPa)
0,4,1,38,2001-01-01,17.0,11.073577,6.123577,8.805244,3.600000,92.971060,...,38.881664,19.211996,30.012077,202.18532,86.166664,24.510103,620.0,1016.5,1006.2,1010.35004
1,4,1,38,2001-01-02,15.0,10.763577,6.703577,8.393994,0.000000,94.317500,...,36.044970,22.668570,29.593008,227.16107,55.375000,327.718200,1942.0,1018.8,1009.3,1012.97090
2,4,1,38,2001-01-03,15.0,9.553577,6.373577,8.138577,2.800000,93.021324,...,37.419224,18.356470,27.256058,215.83910,86.666664,35.062290,862.0,1021.0,1015.5,1018.71674
3,4,1,38,2001-01-04,15.0,12.503577,8.243577,10.381910,0.100000,94.347970,...,33.278100,18.075441,27.357481,225.98854,57.791668,294.457580,1685.0,1020.4,1013.9,1018.04980
4,4,1,38,2001-01-05,16.0,14.563578,7.933577,12.401077,10.000001,95.628430,...,45.435646,26.099379,36.826267,209.32076,96.625000,4.242425,435.0,1018.2,1005.5,1009.74580


In [53]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458361 entries, 0 to 1458360
Data columns (total 27 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   ESTACION                 1458361 non-null  category      
 1   MAGNITUD                 1458361 non-null  category      
 2   TECNICA                  1458361 non-null  category      
 3   DIA/MES/ANO              1458361 non-null  datetime64[ns]
 4   MEDICION_ugm3            1015726 non-null  float64       
 5   TEMPERATURA_MAX (°C)     1458198 non-null  float64       
 6   TEMPERATURA_MIN (°C)     1458198 non-null  float64       
 7   TEMPERATURA_MED (°C)     1458198 non-null  float64       
 8   PRECIPITACIONES (mm)     1458198 non-null  float64       
 9   HUMEDAD_MAX (%)          1458198 non-null  float64       
 10  HUMEDAD_MIN (%)          1458198 non-null  float64       
 11  HUMEDAD_MED (%)          1458198 non-null  float64       
 12  

In [54]:
display(df_final.tail())

,ESTACION,MAGNITUD,TECNICA,DIA/MES/ANO,MEDICION_ugm3,TEMPERATURA_MAX (°C),TEMPERATURA_MIN (°C),TEMPERATURA_MED (°C),PRECIPITACIONES (mm),HUMEDAD_MAX (%),...,VIENTO_MAX_100 (km/h),VIENTO_MIN_100 (km/h),VIENTO_MED_100 (km/h),VIENTO_DIR_100 (°),NUBES (%),HORAS_SOL (min),RADIACION_SOLAR (W/mÂ²),PRESION_MAX (hPa),PRESION_MIN (hPa),PRESION_MED (hPa)
1458356,48,9,47,2025-06-26,NaN,32.203580,14.313578,24.527327,0.0,74.444820,...,9.726665,3.826853,6.660889,153.868990,0.000000,906.4666,8618.0,1020.6,1017.9,1019.15010
1458357,48,9,47,2025-06-27,24.0,35.553577,18.293577,27.409828,0.0,55.928905,...,15.617529,3.396233,8.696938,103.126854,0.012500,906.1333,8633.0,1023.8,1019.6,1021.62910
1458358,48,9,47,2025-06-28,25.0,37.423576,19.893578,28.952332,0.0,49.371860,...,24.387178,2.520000,10.318219,44.487520,7.108334,862.8872,8412.0,1023.9,1018.9,1021.53345
1458359,48,9,47,2025-06-29,NaN,37.433580,22.563578,30.162743,0.4,47.426750,...,34.201893,4.452954,17.080273,22.641602,11.933334,779.5738,8031.0,1021.8,1016.2,1019.42914
1458360,48,9,47,2025-06-30,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
